# Packages

In [1]:
from langchain import hub
from langchain.agents import tool, AgentExecutor, create_structured_chat_agent

# from langchain.agents import AgentExecutor,
from langchain.agents.agent_types import AgentType
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# from langchain_core.messages import , HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# from langchain_core.prompts import MessagesPlaceholder
# from langchain_core.pydantic_v1 import BaseModel
from langchain.pydantic_v1 import BaseModel, Field

from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_fireworks import ChatFireworks
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, tool

import pandas as pd


from dotenv import load_dotenv, find_dotenv
from pprint import pprint


from typing import Optional, Type


# Import things that are needed generically

In [2]:
_ = load_dotenv(find_dotenv())

# Get the data

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Create custom agent

In [5]:
chat = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=64,
    top_k=20,
    top_p=1.0,
)

/usr/local/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_k is not default parameter.
                top_k was transferred to model_kwargs.
                Please confirm that top_k is what you intended.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


In [6]:
tool_calling_chat = ChatFireworks(
    model="accounts/fireworks/models/firefunction-v2",
)

In [7]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

## Defining custom tool

In [8]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


class LengthInput(BaseModel):
    query: str = Field(description="should be a string with one or many words")

In [9]:
class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool"""
        return "LangChain"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously"""
        raise NotImplementedError("custom_search does not support async")

In [10]:
class CustomCalculatorTool(BaseTool):
    name = "calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = False

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool"""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously"""
        raise NotImplementedError("Calculator does not support async")

In [11]:
class CustomLengthTool(BaseTool):
    name = "length"
    description = "Returns the length of a word."
    args_schema: Type[BaseModel] = LengthInput
    return_direct: bool = False

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool"""
        return str(len(query))

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously"""
        raise NotImplementedError("length does not support async")

In [12]:
search = CustomSearchTool()
print(search.name)
print(search.description)
print(search.args)

custom_search
useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}


In [13]:
multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

calculator
useful for when you need to answer questions about math
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}
False


In [14]:
length = CustomLengthTool()
print(length.name)
print(length.description)
print(length.args)
print(length.return_direct)

length
Returns the length of a word.
{'query': {'title': 'Query', 'description': 'should be a string with one or many words', 'type': 'string'}}
False


In [15]:
length.invoke({"query": "today is a beautiful day to learn"})

'33'

In [16]:
search.invoke("today is a beautiful day to learn")

'LangChain'

In [17]:
multiply.invoke({"a": 4, "b": 5})

20

In [18]:
tools = [search, multiply, length]

## Agent creation

In [19]:
llm_with_tools = chat.bind_tools(tools)

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [21]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [22]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
list(
    agent_executor.stream({"input": "How many letters are there in the word 'eudca'?"})
)



> Entering new AgentExecutor chain...
The word "eudca" contains 5 letters.

I can provide information and assistance on a wide range of topics, but I don't have real-time capabilities or access to databases to provide up-to-date information on current events. However, I can help you find the latest news

> Finished chain.


[{'output': 'The word "eudca" contains 5 letters.\n\nI can provide information and assistance on a wide range of topics, but I don\'t have real-time capabilities or access to databases to provide up-to-date information on current events. However, I can help you find the latest news',
  'messages': [AIMessage(content='The word "eudca" contains 5 letters.\n\nI can provide information and assistance on a wide range of topics, but I don\'t have real-time capabilities or access to databases to provide up-to-date information on current events. However, I can help you find the latest news')]}]

In [24]:
list(
    agent_executor.stream({"input": "What's the result when you multiply 17 times 45?"})
)



> Entering new AgentExecutor chain...
I'm here to help answer your questions to the best of my ability! When it comes to mathematical queries, I'm always ready. When you multiply 17 by 45, the result is 765. If you have any other questions or need information on current events, I'

> Finished chain.


[{'output': "I'm here to help answer your questions to the best of my ability! When it comes to mathematical queries, I'm always ready. When you multiply 17 by 45, the result is 765. If you have any other questions or need information on current events, I'",
  'messages': [AIMessage(content="I'm here to help answer your questions to the best of my ability! When it comes to mathematical queries, I'm always ready. When you multiply 17 by 45, the result is 765. If you have any other questions or need information on current events, I'")]}]

## Agent creation with memory

In [25]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [26]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

In [27]:
MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events and if it's possible you always use a tool",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [28]:
chat_history = []

In [29]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, return_intermediate_steps=True
)

In [30]:
input1 = "how many letters are there in the word 'educa'?"

In [31]:
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)



> Entering new AgentExecutor chain...
The word "educa" contains 5 letters. I am a language model AI, and I can provide information and answer questions to the best of my knowledge and ability, but I don't have real-time capabilities or external tools to access current events or real-world data. However, I can certainly help

> Finished chain.


In [32]:
result = agent_executor.invoke(
    {"input": "is that a real word?", "chat_history": chat_history}
)
result



> Entering new AgentExecutor chain...
I apologize for the confusion. The word "educa" is not a standard English word. If you meant to ask about the word "educate," it contains 7 letters. Please let me know if you have any other questions or need information on something else.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content="how many letters are there in the word 'educa'?"),
  AIMessage(content='The word "educa" contains 5 letters. I am a language model AI, and I can provide information and answer questions to the best of my knowledge and ability, but I don\'t have real-time capabilities or external tools to access current events or real-world data. However, I can certainly help')],
 'output': 'I apologize for the confusion. The word "educa" is not a standard English word. If you meant to ask about the word "educate," it contains 7 letters. Please let me know if you have any other questions or need information on something else.',
 'intermediate_steps': []}

In [34]:
input2 = "is that a real word?"
input3 = "What's the result of multiplying 17 times 45?"

chat_history.extend(
    [
        HumanMessage(content=input2),
        AIMessage(content=result["output"]),
    ]
)
result = agent_executor.invoke({"input": input3, "chat_history": chat_history})
result



> Entering new AgentExecutor chain...
The result of multiplying 17 by 45 is 765. I'm here to help with any questions or information you need. Let me know if you have anything else you'd like to know!

> Finished chain.


{'input': "What's the result of multiplying 17 times 45?",
 'chat_history': [HumanMessage(content="how many letters are there in the word 'educa'?"),
  AIMessage(content='The word "educa" contains 5 letters. I am a language model AI, and I can provide information and answer questions to the best of my knowledge and ability, but I don\'t have real-time capabilities or external tools to access current events or real-world data. However, I can certainly help'),
  HumanMessage(content='is that a real word?'),
  AIMessage(content='I apologize for the confusion. The word "educa" is not a standard English word. If you meant to ask about the word "educate," it contains 7 letters. Please let me know if you have any other questions or need information on something else.')],
 'output': "The result of multiplying 17 by 45 is 765. I'm here to help with any questions or information you need. Let me know if you have anything else you'd like to know!",
 'intermediate_steps': []}

## Structured Chat Agent

In [35]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

In [36]:
prompt.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'],
 'kwargs': {'input_variables': ['agent_scratchpad',
   'input',
   'tool_names',
   'tools'],
  'input_types': {'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]},
  'metadata': {'lc_hub_owner': 'hwchase17',
   'lc_hub_repo': 'structured-chat-agent',
   'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'},
  'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name)

In [37]:
list(prompt.to_json()["kwargs"]["messages"][0])[0][1]

PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Obse

In [41]:
agent = create_structured_chat_agent(llm_with_tools, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [42]:
agent_executor.invoke({"input": "How many letters are there in the word 'eudca'?"})



> Entering new AgentExecutor chain...
Thought: The user wants to know the number of letters in the word 'eudca'. I can use the 'length' tool to answer this question.

Action:
```
{
  "action": "length",
  "action_input": {
    "query": "e1Could not parse LLM output: The user wants to know the number of letters in the word 'eudca'. However, the word provided in the observation is incomplete. I will assume the complete word is 'eudca' and use the 'length' tool to answer this question.

Action:
```
{
 Invalid or incomplete responseAction:
```
{
  "action": "length",
  "action_input": {
    "query": "eudca"
  }
}
```5Action:
```
{
  "action": "Final Answer",
  "action_input": "The word 'eudca' contains 5 letters."
}
```

> Finished chain.


{'input': "How many letters are there in the word 'eudca'?",
 'output': "The word 'eudca' contains 5 letters."}

# Experiments

In [23]:
class ExtractFields(BaseModel):
    name: str
    age: int


chat = ChatFireworks(
    model="accounts/fireworks/models/firefunction-v2",
).bind_tools([ExtractFields])

result = chat.invoke("I am a 27 year old named Erick")

pprint(result.additional_kwargs["tool_calls"][0])

{'function': {'arguments': '{"name": "Erick", "age": 27}',
              'name': 'ExtractFields'},
 'id': 'call_cYfcB3JTTirtl6vXiRBumbFA',
 'index': 0,
 'type': 'function'}


In [25]:
chat = ChatFireworks(
    model="accounts/fireworks/models/firefunction-v2",
)

In [30]:
llm = ChatOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key="YOUR_FW_API_KEY",
    model="accounts/fireworks/models/firefunction-v1",
    temperature=0.0,
    max_tokens=256,
)

In [54]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [56]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [57]:
tool.name

'wikipedia'

In [58]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [59]:
tool.args

{'query': {'title': 'Query',
  'description': 'query to look up on wikipedia',
  'type': 'string'}}

In [60]:
tool.return_direct

False

In [61]:
tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [62]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [63]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

In [64]:
tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

In [65]:
tool.name

'wiki-tool'

In [66]:
tool.description

'look up things in wikipedia'